In [6]:
import numpy as np
import pickle
from common.functions import * 

### Q1.Softmax 함수 구현하기

In [7]:
def softmax(x):
    ## 2차원인 경우와 1차원인 경우로 나누어서 계산
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis = 0)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)
        return y.T
    
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

### Q3.3층 신경망 구현(p.89)

In [8]:
# 1) def init_network()에 가중치와 편향을 저장하도록 network 딕셔너리를 이용하여 표현하세요. 가중치는 임의의 숫자를 넣어주세요. 함수의 원형은 아래와 같고, network를 반환하도록 설계하세요.(return network)

#교재
""" 
def init_network(): 
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])
    return network
"""

def init_network(): ## 가중치와 편향 저장
    network = {}
    network['W1'] = np.random.randn(2, 2) ## 입력층 to 첫번째 은닉층 가중치
    network['b1'] = np.random.randn(2) ## 첫번째 은닉층(h1, h2)에 들어가는 편향 2개
    network['W2'] = np.random.randn(2, 3) ## 첫번째 은닉층 to 두번째 은닉층 가중치
    network['b2'] = np.random.randn(3) ## 두번째 은닉층(s1, s2, s3)에 들어가는 가중치
    network['W3'] = np.random.randn(3, 3) ## 두번째 은닉층 to 출력층 가중치
    network['b3'] = np.random.randn(3) ## 출력층에 들어가는 가중치
    return network

In [9]:
# 2) def forward(network, x)에서 입력 신호를 출력으로 변환하는 처리 과정을 구현해 보려고 합니다. 1)에서 반환한 network를 forward의 입력 인수로 넣는 것을 고려해서 구현하세요. 이 함수의 출력값은 y입니다.(return y)
def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    return y

In [13]:
# 3) 이제 main 함수에서 아래와 같은 형태를 실행하고, 값의 형태가 올바른 지 확인하세요.
network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)  # [ 0.31682708  0.69627909]
print(y)

[-1.46195347  2.14950918 -1.1838769 ]


### Q4. MNIST 데이터를 활용하여 신경망의 추론처리 과정을 구현하기 (p.96 ~ p.105)

In [14]:
# 1) 아래 코드를 실행하여서 라이브러리를 import 하세요
import sys,os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train),(x_test, t_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [15]:
#2) 이제부터 MNIST 데이터셋을 가지고 추론을 수행하는 신경망을 구현해 보려고 합니다. 아래 세 가지 함수에 조건에 맞게 구현하세요.

#a. def get_data() : mnist.py 의 load_mnist 함수를 이용해서 데이터를 불러온 후, 테스트 데이터를 반환하도록 구현하세요. (return x_test, t_test)
def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test

#b. def init_network() : 기존에 다운받은 mnist.pkl 에는 학습이 완료된 가중치 매개변수가 들어 있습니다. Mnist.pkl 를 불러온 다음에 가중치 매개변수를 반환하도록 구현하세요. (return network)
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        # 학습된 가중치 매개변수가 담긴 파일
        # 학습 없이 바로 추론을 수행
        network = pickle.load(f)
    return network

#c. def predict(network, x) : b 에서 불러온 가중치 매개변수와 a 에서 불러온 데이터를 활용해서 연산을 진행한 후, 각 레이블의 확률을 반환하도록 구현하세요. 해당 신경망의 구조는 아래의 형태입니다. (return y)
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3) #없이해도 결과 동일
    return y

#d. def init_network2() : b 에서와 다르게, Q3 에서 구현했던 3 층 신경망과 비슷한 형태로 network 를 만들어서 반환하도록 구현하세요. 가중치는 무작위로 초기화를 시켜주시면 됩니다. (return network)
def init_network2():
    network = {}
    network['W1'] = np.random.ranf((784, 50))
    network['b1'] = np.random.ranf((50,))
    network['W2'] = np.random.ranf((50, 100))
    network['b2'] = np.random.ranf((100,))
    network['W3'] = np.random.ranf((100, 10))
    network['b3'] = np.random.ranf((10,))
    return network

In [16]:
#e. b 에서 구현한 네트워크와 d 에서 구현한 네트워크와의 정확도를 비교해보고, 어떤 점에서 차이가 있었는 지를 추론해 보세요.
x, t = get_data()
network = init_network()
accuracy_cnt = 0 #맞힌 개수
batch_size = 100

# pickle파일의 가중치
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)  # 확률이 가장 높은 원소의 인덱스를 얻는다.(마지막 softmax된 확률 중)
    if p == t[i]:
        accuracy_cnt += 1
print("pickle파일의 Accuracy:" + str(float(accuracy_cnt) / len(x)))  # Accuracy:0.9352

# 랜덤 가중치
network = init_network2()
accuracy_cnt = 0 #초기화

for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)  # 확률이 가장 높은 원소의 인덱스를 얻는다.(마지막 softmax된 확률 중)
    if p == t[i]:
        accuracy_cnt += 1
print("랜덤 가중치 Accuracy:" + str(float(accuracy_cnt) / len(x)))

"""
- 사전 훈련된 가중치를 사용한것은 딥러닝과정을 통해 구현할 목표(마지막 가중치)
- 랜덤값을 사용한 것은 역전파과정을 진행하기 전 처음 가중치
"""

pickle파일의 Accuracy:0.9352
랜덤 가중치 Accuracy:0.0974


'\n- 사전 훈련된 가중치를 사용한것은 딥러닝과정을 통해 구현할 목표(마지막 가중치)\n- 랜덤값을 사용한 것은 역전파과정을 진행하기 전 처음 가중치\n'

In [17]:
# 3) 이전에는 데이터 하나씩 계산을 해서 결과를 출력하는 과정이었습니다. 이제는 데이터를 여러 개 이상 사용해서 한 번에 연산을 진행해보려고 합니다. 한번에 100 개씩 처리할 수 있도록 main 에서 구현하세요. (p.104)
def main():
    x, t = get_data()
    network = init_network()
    accuracy_cnt = 0 #맞힌 개수

    batch_size = 100

    for i in range(0, len(x), batch_size):
        x_batch = x[i:i+batch_size]
        y_batch = predict(network, x_batch)
        p = np.argmax(y_batch, axis=1) #마지막 10개를 sofamax통과시킨 것들 중 최대값(확률)
        accuracy_cnt += np.sum(p == t[i:i+batch_size]) #제일높아보이는 확률과, 라벨값이 일치하는 개수

    print("Accuracy:" + str(float(accuracy_cnt) / len(x)))  # Accuracy:0.9352

In [18]:
main()

Accuracy:0.9352
